In [ ]:
import getpass
import math
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt

from urllib.parse import quote_plus

from kgforge.core import KnowledgeGraphForge

In [ ]:
# stats = forge.search({"type": "ElasticSearchViewStatistics"})
# forge.deprecate(stats)

In [ ]:
# stats = forge.search({"type": "SimilarityBoostingData"})
# forge.deprecate(stats)

# Helpers

In [ ]:
Statistics = namedtuple('Statistics', 'min max mean std')

In [ ]:
def set_elastic_view(forge, view):
    forge._store.service.elastic_endpoint["endpoint"] = "/".join(
        (VIEWS_ENDPOINT, quote_plus(view), "_search"))


def get_all_vectors(forge, resource_limit):
    all_embeddings = forge.elastic(f"""{{
        "from" : 0,
        "size" : {resource_limit},
        "query": {{
            "term": {{"_deprecated": false}}
        }}
    }}
    """)
    vectors = {
        result._source["@id"]: result._source["embedding"]
        for result in all_embeddings
    }
    return vectors


def get_all_scores(forge, vectors, formula, param_name, resource_limit=200, boosting=None):
    score_values = set()
    for k, vector in vectors.items():
        query = f"""{{
          "size": {len(vectors)},
          "query": {{
            "script_score": {{
                "query": {{
                    "bool" : {{
                      "must_not" : {{
                        "term" : {{ "@id": "{k}" }}
                      }},
                      "must": {{ "exists": {{ "field": "embedding" }} }}
                    }}
                }},
                "script": {{
                    "source": "{formula}",
                    "params": {{
                      "{param_name}": {vector}
                    }}
                }}
            }}
          }}
        }}"""

        res = forge.elastic(query)
        for el in res:
            boost_factor = 1
            if boosting:
                boost_factor = 1 + boosting[el._source["@id"]]
            score_values.add(el._score * boost_factor)
    score_values = np.array(list(score_values))
    return score_values


def get_view_stats(forge, vectors, formula, param_name, resource_limit=200, boosting=None):
    score_values = get_all_scores(
        forge, vectors, formula, param_name, resource_limit, boosting)
    return score_values, Statistics(
        score_values.min(),
        score_values.max(),
        score_values.mean(),
        score_values.std())


def register_stats(forge, view_id, sample_size, stats, formula, param_name, boosting_resource=None):
    json_data = {
        "type": "ElasticSearchViewStatistics",
        "scriptScore": formula,
        "vectorParameter": param_name,
        "series": [
            {
              "statistic": "min",
              "unitCode": "dimensionless",
              "value": stats.min
            },
            {
              "statistic": "max",
              "unitCode": "dimensionless",
              "value": stats.max
            },
            {
              "statistic": "mean",
              "unitCode": "dimensionless",
              "value": stats.mean
            },
            {
              "statistic": "standard deviation",
              "unitCode": "dimensionless",
              "value": stats.std
            },
            {
              "statistic": "N",
              "unitCode": "dimensionless",
              "value": sample_size
            }
        ],
        "derivation": {
            "type": "Derivation",
            "entity": {
                "id": view_id
            }
        }
    }
    if boosting_resource:
        json_data["boosted"] = True
        json_data["generation"] = {
            "type": "Generation",
            "activity": {
                "used": {
                    "id": boosting_resource
                }
            }
        }
    else:
        json_data["boosted"] = False
    stats_resource = forge.from_json(json_data)
    forge.register(stats_resource)
    
    
def get_score_deviation(forge, point_id, vector, k, formula, param_name):
    query = f"""{{
      "size": {k},
      "query": {{
        "script_score": {{
          "query": {{
                "exists": {{
                    "field": "embedding"
                }}
          }},
          "script": {{
            "source": "{formula}",
            "params": {{
              "{param_name}": {vector}
            }}
          }}
        }}
      }}
    }}"""

    result = forge.elastic(query)
    scores = set()
    for el in result:
        if point_id != el._source["@id"]:
            scores.add(el._score)
    scores = np.array(list(scores))
    return math.sqrt(((1 - scores)**2).mean())


def register_boosting_data(forge, view_id, deviation, formula, param_name):
    factors = [
        {"entity": {"@id": k}, "value": 1 + v, "unitCode": "dimensionless"}
        for k, v in data.items()
    ]
    resource = forge.from_json({
        "type": "SimilarityBoostingSeries",
        "scriptScore": formula,
        "vectorParameter": param_name,
        "series": factors,
        "derivation": {
            "type": "Derivation",
            "entity": {
                "id": view_id
            }
        }
    })
    forge.register(resource)
    return resource.id

# Configure forge

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
ENDPOINT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"
ORG = "dke"
PROJECT = "seu-embeddings"
VIEWS_ENDPOINT = "/".join(
    (ENDPOINT, "views", quote_plus(ORG), quote_plus(PROJECT)))

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN, 
                            endpoint=ENDPOINT,        
                            bucket=f"{ORG}/{PROJECT}")

# Get all embedding vectors

In [ ]:
vector_parameter = "query_vector"

In [ ]:
views = {
    "https://bbp.epfl.ch/neurosciencegraph/data/views/es/dendrite-coprojection-embeddings-view": "cosine_similarity",
    "https://bbp.epfl.ch/neurosciencegraph/data/views/es/axon-coprojection-embeddings-view": "cosine_similarity",
    "https://bbp.epfl.ch/neurosciencegraph/data/views/es/neurite-features-view": "l2_similarity"
}

In [ ]:
formulas = {
    "cosine_similarity": "(cosineSimilarity(params.query_vector, doc['embedding']) + 1.0) / 2",
    "l2_similarity": "1 / (1 + l2norm(params.query_vector, doc['embedding']))"
}

In [ ]:
vectors = {}
for view in views:
    set_elastic_view(forge, view)
    vectors[view] = get_all_vectors(forge, 200)

# Compute and register stats of ES indices

In [ ]:
global_stats = {}
for view in views:
    set_elastic_view(forge, view)
    formula = formulas[views[view]]
    values, stats = get_view_stats(forge, vectors[view], formula, vector_parameter, 200)
    global_stats[view] = values
    register_stats(forge, view, values.shape[0], stats, formula, vector_parameter)

In [ ]:
for view in views:
    print("View ", view)
    plt.hist(
        global_stats[view],
        bins=100)
    plt.title("Distribution of similarities")
    plt.show()

# Compute and register boosting factors for different points

In [ ]:
deviations = {}
for view in views:
    forge._store.service.elastic_endpoint["endpoint"] = "/".join(
        (VIEWS_ENDPOINT, quote_plus(view), "_search"))
    deviations[view] = {}
    for point_id, vector in vectors[view].items():
        deviations[view][point_id] = get_score_deviation(
            forge, point_id, vector, 10, formulas[views[view]], vector_parameter)

In [ ]:
boosting_resources = {}
for view, data in deviations.items():
    boosting_resources[view] = register_boosting_data(
        forge, view, data, formulas[views[view]], vector_parameter)

In [ ]:
boosted_global_stats = {}
for view in views:
    forge._store.service.elastic_endpoint["endpoint"] = "/".join(
        (VIEWS_ENDPOINT, quote_plus(view), "_search"))
    formula = formulas[views[view]]
    boosted_global_stats[view], stats = get_view_stats(
        forge, vectors[view], formula, vector_parameter, 200, deviations[view])
    register_stats(
        forge, view, values.shape[0], stats,
        formula, vector_parameter,
        boosting_resource=boosting_resources[view])

In [ ]:
for view in views:
    print("View ", view)
    plt.hist(
        boosted_global_stats[view],
        bins=100)
    plt.title("Distribution of similarities")
    plt.show()